<a href="https://colab.research.google.com/github/vdpyz/TomatoAppleClassifier/blob/main/AppleTomatosClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Tomato or apple - classification**

dataset (not rgb files removed): https://www.kaggle.com/datasets/samuelcortinhas/apples-or-tomatoes-image-classification

based on: https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

Dominika Wegrzyniak, 07.01.2023

In [25]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torchvision.transforms import ToTensor
from glob import glob                       # pobranie zawartości katalogu
from torch.utils.data import Dataset, DataLoader
from PIL import Image                       # wczytanie pliku graficznego

In [16]:
#change to gpu
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [17]:
#!wget https://github.com/vdpyz/TomatoAppleClassifier/blob/main/archive.zip

--2023-01-07 14:57:58--  https://github.com/vdpyz/TomatoAppleClassifier/blob/main/archive.zip
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘archive.zip’

archive.zip             [ <=>                ] 139.89K  --.-KB/s    in 0.02s   

2023-01-07 14:57:58 (6.08 MB/s) - ‘archive.zip’ saved [143243]



In [19]:
!unzip archive

Archive:  archive.zip
   creating: archive/
   creating: archive/test/
   creating: archive/test/apples/
  inflating: archive/test/apples/img_p1_111.jpeg  
  inflating: archive/test/apples/img_p1_121.jpeg  
  inflating: archive/test/apples/img_p1_127.jpeg  
  inflating: archive/test/apples/img_p1_131.jpeg  
  inflating: archive/test/apples/img_p1_133.jpeg  
  inflating: archive/test/apples/img_p1_139.jpeg  
  inflating: archive/test/apples/img_p1_14.jpeg  
  inflating: archive/test/apples/img_p1_143.jpeg  
  inflating: archive/test/apples/img_p1_15.jpeg  
  inflating: archive/test/apples/img_p1_16.jpeg  
  inflating: archive/test/apples/img_p1_29.jpeg  
  inflating: archive/test/apples/img_p1_30.jpeg  
  inflating: archive/test/apples/img_p1_33.jpeg  
  inflating: archive/test/apples/img_p1_34.jpeg  
  inflating: archive/test/apples/img_p1_36.jpeg  
  inflating: archive/test/apples/img_p1_50.jpeg  
  inflating: archive/test/apples/img_p1_52.jpeg  
  inflating: archive/test/apples/img_p

Prepare images to ML
Sizes of images are different -> set all to [100, 100]

In [26]:
class CustomDataset(Dataset):
    def __init__(self, root_dir):
        self.transform = ToTensor()
        classes = glob(f"{root_dir}/*") # pobieramy listę katalogów poszczególnych klas
        self.class_ids = []     # lista indeksów klas
        self.filenames = []     # lista ścieżek i nazw plików

        # generujemy listę plików i indeksów klas
        for id, class_dir in enumerate(classes):
            files = glob(f"{class_dir}/*.jpeg")
            self.filenames.extend(files)    # rozszerzamy listę, a więc extend, a nie append
            self.class_ids.extend([id]*len(files))  # hint: [5]*3 wygeneruje listę [5, 5, 5]

    def __len__(self):
        return len(self.class_ids)

    def __getitem__(self, idx):
        image = Image.open(self.filenames[idx])
        new_image = image.resize((100, 100))
        return self.transform(new_image), self.class_ids[idx]

In [27]:
train_ds = CustomDataset('archive/train')
test_ds = CustomDataset('archive/test')
print(len(train_ds))
print(len(test_ds))

290
97


In [28]:
train_dl = DataLoader(train_ds, batch_size=5000, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=5000, shuffle=True)

In [30]:
for X, y in test_dl:
    print(X.shape, y.shape)

torch.Size([97, 3, 100, 100]) torch.Size([97])
